In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '../')

In [2]:
from sklearn.preprocessing import StandardScaler

In [3]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [4]:
data = pd.read_pickle('../ts_data/grid')

In [5]:
data.head()

,power
date_str,
2020-02-13 07:00:00,12.82
2020-02-13 08:00:00,8.69
2020-02-13 09:00:00,13068.87
2020-02-13 10:00:00,2788.11
2020-02-13 11:00:00,131494.69


We will now split in to train, val and test

In [6]:
def get_lags(dataframe):
    dfs = []
    for i in range(1, 14*2 +1):
        df1 = dataframe.shift(i)
        df1.columns = [col + "_" + str(-i) for col in dataframe.columns]
        dfs.append(df1.iloc[:,0:])
    dfs.append(dataframe)
    new_df = pd.concat(dfs, axis=1).dropna()
    return new_df

In [7]:
def get_output(dataframe):
    dfs = [dataframe]
    for i in range(1, 14):
        df1 = dataframe[['power']].shift(-i)
        df1.columns = [col + "_" + str(i) for col in ['power']]
        dfs.append(df1.iloc[:,0:])
    return pd.concat(dfs, axis=1).dropna()

In [8]:
train = data[0:-14 * (73 + 36)]

In [9]:
val = data[:-14*36]

In [10]:
# scaling the data
scaler = StandardScaler()
scaler.fit(train.values)

StandardScaler()

In [11]:
train_array = scaler.transform(train.values)

In [12]:
train_df = pd.DataFrame(train_array, columns=data.columns)

In [13]:
val_array = scaler.transform(val.values)
val_df = pd.DataFrame(val_array, columns=data.columns)

In [14]:
lag_train = get_lags(train_df)
train_output = get_output(lag_train)

In [15]:
train_output

,power_-1,power_-2,power_-3,power_-4,power_-5,power_-6,power_-7,power_-8,power_-9,power_-10,...,power_4,power_5,power_6,power_7,power_8,power_9,power_10,power_11,power_12,power_13
28,-0.086938,0.987730,1.775831,1.930620,2.355474,2.393521,2.298017,1.921558,1.283832,0.439106,...,0.133838,0.658852,1.095236,1.412690,1.489767,1.437742,0.773688,0.205039,-0.215197,-0.655419
29,-1.105681,-0.086938,0.987730,1.775831,1.930620,2.355474,2.393521,2.298017,1.921558,1.283832,...,0.658852,1.095236,1.412690,1.489767,1.437742,0.773688,0.205039,-0.215197,-0.655419,-1.105733
30,-1.105711,-1.105681,-0.086938,0.987730,1.775831,1.930620,2.355474,2.393521,2.298017,1.921558,...,1.095236,1.412690,1.489767,1.437742,0.773688,0.205039,-0.215197,-0.655419,-1.105733,-1.105750
31,-1.009594,-1.105711,-1.105681,-0.086938,0.987730,1.775831,1.930620,2.355474,2.393521,2.298017,...,1.412690,1.489767,1.437742,0.773688,0.205039,-0.215197,-0.655419,-1.105733,-1.105750,-0.989981
32,-0.430042,-1.009594,-1.105711,-1.105681,-0.086938,0.987730,1.775831,1.930620,2.355474,2.393521,...,1.489767,1.437742,0.773688,0.205039,-0.215197,-0.655419,-1.105733,-1.105750,-0.989981,-0.564970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3594,1.131678,0.905539,2.001347,1.942464,1.496141,0.835891,-0.009777,-0.772938,-1.088316,-1.105739,...,-1.105761,-1.090326,-0.790664,-0.128751,0.788586,1.431232,1.912733,2.182486,1.756493,1.880022
3595,1.333218,1.131678,0.905539,2.001347,1.942464,1.496141,0.835891,-0.009777,-0.772938,-1.088316,...,-1.090326,-0.790664,-0.128751,0.788586,1.431232,1.912733,2.182486,1.756493,1.880022,0.732016
3596,1.273818,1.333218,1.131678,0.905539,2.001347,1.942464,1.496141,0.835891,-0.009777,-0.772938,...,-0.790664,-0.128751,0.788586,1.431232,1.912733,2.182486,1.756493,1.880022,0.732016,0.509837
3597,0.103523,1.273818,1.333218,1.131678,0.905539,2.001347,1.942464,1.496141,0.835891,-0.009777,...,-0.128751,0.788586,1.431232,1.912733,2.182486,1.756493,1.880022,0.732016,0.509837,-0.054253


In [16]:
lag_val = get_lags(val_df)
val_output = get_output(lag_val)

In [17]:
val_output = val_output[train_output.index[-1] - train_output.index[0]+1:]

In [18]:
val_output

,power_-1,power_-2,power_-3,power_-4,power_-5,power_-6,power_-7,power_-8,power_-9,power_-10,...,power_4,power_5,power_6,power_7,power_8,power_9,power_10,power_11,power_12,power_13
3599,-1.105761,-0.635850,0.103523,1.273818,1.333218,1.131678,0.905539,2.001347,1.942464,1.496141,...,1.431232,1.912733,2.182486,1.756493,1.880022,0.732016,0.509837,-0.054253,-0.721539,-1.105766
3600,-1.090326,-1.105761,-0.635850,0.103523,1.273818,1.333218,1.131678,0.905539,2.001347,1.942464,...,1.912733,2.182486,1.756493,1.880022,0.732016,0.509837,-0.054253,-0.721539,-1.105766,-1.082508
3601,-0.790664,-1.090326,-1.105761,-0.635850,0.103523,1.273818,1.333218,1.131678,0.905539,2.001347,...,2.182486,1.756493,1.880022,0.732016,0.509837,-0.054253,-0.721539,-1.105766,-1.082508,-0.762958
3602,-0.128751,-0.790664,-1.090326,-1.105761,-0.635850,0.103523,1.273818,1.333218,1.131678,0.905539,...,1.756493,1.880022,0.732016,0.509837,-0.054253,-0.721539,-1.105766,-1.082508,-0.762958,-0.094757
3603,0.788586,-0.128751,-0.790664,-1.090326,-1.105761,-0.635850,0.103523,1.273818,1.333218,1.131678,...,1.880022,0.732016,0.509837,-0.054253,-0.721539,-1.105766,-1.082508,-0.762958,-0.094757,0.406350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4616,2.645486,2.657666,2.524155,2.103241,1.336195,0.764087,-0.081379,-0.820566,-1.078566,-1.105752,...,-1.105729,-1.084446,-0.830310,-0.006744,0.736865,1.417084,2.052106,2.093953,1.752583,2.247351
4617,2.437631,2.645486,2.657666,2.524155,2.103241,1.336195,0.764087,-0.081379,-0.820566,-1.078566,...,-1.084446,-0.830310,-0.006744,0.736865,1.417084,2.052106,2.093953,1.752583,2.247351,2.311251
4618,1.980177,2.437631,2.645486,2.657666,2.524155,2.103241,1.336195,0.764087,-0.081379,-0.820566,...,-0.830310,-0.006744,0.736865,1.417084,2.052106,2.093953,1.752583,2.247351,2.311251,1.831690
4619,1.205958,1.980177,2.437631,2.645486,2.657666,2.524155,2.103241,1.336195,0.764087,-0.081379,...,-0.006744,0.736865,1.417084,2.052106,2.093953,1.752583,2.247351,2.311251,1.831690,1.183749


In [20]:
train_x, train_y = train_output.iloc[:, 0:-14], train_output.iloc[:, -14:]

In [22]:
train_y

,power,power_1,power_2,power_3,power_4,power_5,power_6,power_7,power_8,power_9,power_10,power_11,power_12,power_13
28,-1.105681,-1.105711,-1.009594,-0.430042,0.133838,0.658852,1.095236,1.412690,1.489767,1.437742,0.773688,0.205039,-0.215197,-0.655419
29,-1.105711,-1.009594,-0.430042,0.133838,0.658852,1.095236,1.412690,1.489767,1.437742,0.773688,0.205039,-0.215197,-0.655419,-1.105733
30,-1.009594,-0.430042,0.133838,0.658852,1.095236,1.412690,1.489767,1.437742,0.773688,0.205039,-0.215197,-0.655419,-1.105733,-1.105750
31,-0.430042,0.133838,0.658852,1.095236,1.412690,1.489767,1.437742,0.773688,0.205039,-0.215197,-0.655419,-1.105733,-1.105750,-0.989981
32,0.133838,0.658852,1.095236,1.412690,1.489767,1.437742,0.773688,0.205039,-0.215197,-0.655419,-1.105733,-1.105750,-0.989981,-0.564970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3594,1.333218,1.273818,0.103523,-0.635850,-1.105761,-1.090326,-0.790664,-0.128751,0.788586,1.431232,1.912733,2.182486,1.756493,1.880022
3595,1.273818,0.103523,-0.635850,-1.105761,-1.090326,-0.790664,-0.128751,0.788586,1.431232,1.912733,2.182486,1.756493,1.880022,0.732016
3596,0.103523,-0.635850,-1.105761,-1.090326,-0.790664,-0.128751,0.788586,1.431232,1.912733,2.182486,1.756493,1.880022,0.732016,0.509837
3597,-0.635850,-1.105761,-1.090326,-0.790664,-0.128751,0.788586,1.431232,1.912733,2.182486,1.756493,1.880022,0.732016,0.509837,-0.054253


In [23]:
train_x_val = train_x.values.reshape(train_x.shape[0], 1, train_x.shape[1])

In [24]:
train_x_val.shape

(3571, 1, 28)

In [25]:
train_y = train_y.values

In [26]:
train_y.shape

(3571, 14)

In [27]:
val_x, val_y = val_output.iloc[:, 0:14*2], val_output.iloc[:, 14*2:]

In [28]:
val_y = val_y.values

In [29]:
val_x_values = val_x.values.reshape(val_x.shape[0], 1, val_x.shape[1])

In [30]:
val_nn = (val_x_values, val_y)

In [31]:
val_nn

(array([[[-1.10576133, -0.6358504 ,  0.10352251, ..., -0.0269315 ,
          -0.76535479, -1.08848774]],
 
        [[-1.09032645, -1.10576133, -0.6358504 , ...,  0.81268119,
          -0.0269315 , -0.76535479]],
 
        [[-0.79066372, -1.09032645, -1.10576133, ...,  1.46130853,
           0.81268119, -0.0269315 ]],
 
        ...,
 
        [[ 1.9801768 ,  2.43763087,  2.64548567, ..., -1.10573001,
           0.23211939,  1.25732529]],
 
        [[ 1.20595782,  1.9801768 ,  2.43763087, ..., -1.07944975,
          -1.10573001,  0.23211939]],
 
        [[-0.07760403,  1.20595782,  1.9801768 , ..., -0.80968123,
          -1.07944975, -1.10573001]]]),
 array([[-1.09032645, -0.79066372, -0.12875085, ..., -0.05425323,
         -0.721539  , -1.10576579],
        [-0.79066372, -0.12875085,  0.78858594, ..., -0.721539  ,
         -1.10576579, -1.08250813],
        [-0.12875085,  0.78858594,  1.43123214, ..., -1.10576579,
         -1.08250813, -0.76295828],
        ...,
        [ 1.20595782, -0

We'll try to fit an RNN and see

In [32]:
import tensorflow as tf

model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(32, return_sequences=False, input_shape = (train_x_val.shape[1], train_x_val.shape[2])))
model.add(tf.keras.layers.Dense(14))

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                7808      
_________________________________________________________________
dense (Dense)                (None, 14)                462       
Total params: 8,270
Trainable params: 8,270
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.compile(loss=tf.losses.MeanSquaredError(),
                          optimizer=tf.optimizers.Adam(0.01),
                          metrics=[tf.metrics.MeanAbsoluteError()])

In [35]:
history = model.fit(train_x_val, train_y, epochs =10, validation_data = val_nn)

Epoch 1/10
112/112 [==============================] - 3s 10ms/step - loss: 0.3727 - mean_absolute_error: 0.4609 - val_loss: 0.2785 - val_mean_absolute_error: 0.4213
Epoch 2/10
112/112 [==============================] - 0s 3ms/step - loss: 0.1635 - mean_absolute_error: 0.2780 - val_loss: 0.2693 - val_mean_absolute_error: 0.4033
Epoch 3/10
112/112 [==============================] - 0s 3ms/step - loss: 0.1533 - mean_absolute_error: 0.2668 - val_loss: 0.2674 - val_mean_absolute_error: 0.4062
Epoch 4/10
112/112 [==============================] - 0s 3ms/step - loss: 0.1467 - mean_absolute_error: 0.2586 - val_loss: 0.2449 - val_mean_absolute_error: 0.3912
Epoch 5/10
112/112 [==============================] - 0s 3ms/step - loss: 0.1514 - mean_absolute_error: 0.2619 - val_loss: 0.2408 - val_mean_absolute_error: 0.3806
Epoch 6/10
112/112 [==============================] - 0s 2ms/step - loss: 0.1475 - mean_absolute_error: 0.2576 - val_loss: 0.2428 - val_mean_absolute_error: 0.3898
Epoch 7/10
112/

In [67]:
# history.history

In [68]:
# let's check the test data

In [36]:
test = data

In [37]:
test_array = scaler.transform(test.values)
test_df = pd.DataFrame(test_array, columns=data.columns)

In [38]:
test_lags = get_lags(test_df)
test_output = get_output(test_lags)

In [39]:
test_output = test_output[val_output.index[-1] - train_output.index[0]+1: ]

In [40]:
test_output

,power_-1,power_-2,power_-3,power_-4,power_-5,power_-6,power_-7,power_-8,power_-9,power_-10,...,power_4,power_5,power_6,power_7,power_8,power_9,power_10,power_11,power_12,power_13
4621,-1.105729,-0.077604,1.205958,1.980177,2.437631,2.645486,2.657666,2.524155,2.103241,1.336195,...,1.417084,2.052106,2.093953,1.752583,2.247351,2.311251,1.831690,1.183749,0.164669,-1.105684
4622,-1.084446,-1.105729,-0.077604,1.205958,1.980177,2.437631,2.645486,2.657666,2.524155,2.103241,...,2.052106,2.093953,1.752583,2.247351,2.311251,1.831690,1.183749,0.164669,-1.105684,-1.072648
4623,-0.830310,-1.084446,-1.105729,-0.077604,1.205958,1.980177,2.437631,2.645486,2.657666,2.524155,...,2.093953,1.752583,2.247351,2.311251,1.831690,1.183749,0.164669,-1.105684,-1.072648,-0.881453
4624,-0.006744,-0.830310,-1.084446,-1.105729,-0.077604,1.205958,1.980177,2.437631,2.645486,2.657666,...,1.752583,2.247351,2.311251,1.831690,1.183749,0.164669,-1.105684,-1.072648,-0.881453,-0.151633
4625,0.736865,-0.006744,-0.830310,-1.084446,-1.105729,-0.077604,1.205958,1.980177,2.437631,2.645486,...,2.247351,2.311251,1.831690,1.183749,0.164669,-1.105684,-1.072648,-0.881453,-0.151633,0.304814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5120,2.652151,2.656393,2.471986,1.979627,1.322296,0.450078,-0.460306,-0.960210,-1.105745,-1.105746,...,-1.105745,-1.105742,-0.976812,-0.375374,0.579614,0.955157,1.405182,1.704691,1.817287,1.896027
5121,2.492696,2.652151,2.656393,2.471986,1.979627,1.322296,0.450078,-0.460306,-0.960210,-1.105745,...,-1.105742,-0.976812,-0.375374,0.579614,0.955157,1.405182,1.704691,1.817287,1.896027,1.607542
5122,2.060756,2.492696,2.652151,2.656393,2.471986,1.979627,1.322296,0.450078,-0.460306,-0.960210,...,-0.976812,-0.375374,0.579614,0.955157,1.405182,1.704691,1.817287,1.896027,1.607542,1.891312
5123,1.288239,2.060756,2.492696,2.652151,2.656393,2.471986,1.979627,1.322296,0.450078,-0.460306,...,-0.375374,0.579614,0.955157,1.405182,1.704691,1.817287,1.896027,1.607542,1.891312,1.144920


In [41]:
test_x, test_y = test_output.iloc[:, 0:14*2], test_output.iloc[:, 14*2:]

In [42]:
test_x

,power_-1,power_-2,power_-3,power_-4,power_-5,power_-6,power_-7,power_-8,power_-9,power_-10,...,power_-19,power_-20,power_-21,power_-22,power_-23,power_-24,power_-25,power_-26,power_-27,power_-28
4621,-1.105729,-0.077604,1.205958,1.980177,2.437631,2.645486,2.657666,2.524155,2.103241,1.336195,...,2.425758,2.637554,2.691235,2.566129,2.232068,1.655391,0.848909,-0.057531,-0.809681,-1.079450
4622,-1.084446,-1.105729,-0.077604,1.205958,1.980177,2.437631,2.645486,2.657666,2.524155,2.103241,...,1.956210,2.425758,2.637554,2.691235,2.566129,2.232068,1.655391,0.848909,-0.057531,-0.809681
4623,-0.830310,-1.084446,-1.105729,-0.077604,1.205958,1.980177,2.437631,2.645486,2.657666,2.524155,...,1.237017,1.956210,2.425758,2.637554,2.691235,2.566129,2.232068,1.655391,0.848909,-0.057531
4624,-0.006744,-0.830310,-1.084446,-1.105729,-0.077604,1.205958,1.980177,2.437631,2.645486,2.657666,...,0.190598,1.237017,1.956210,2.425758,2.637554,2.691235,2.566129,2.232068,1.655391,0.848909
4625,0.736865,-0.006744,-0.830310,-1.084446,-1.105729,-0.077604,1.205958,1.980177,2.437631,2.645486,...,-1.105752,0.190598,1.237017,1.956210,2.425758,2.637554,2.691235,2.566129,2.232068,1.655391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5120,2.652151,2.656393,2.471986,1.979627,1.322296,0.450078,-0.460306,-0.960210,-1.105745,-1.105746,...,0.496968,0.220211,-0.348993,-0.996680,-1.105747,-1.105752,-0.216993,0.826037,1.606459,1.705478
5121,2.492696,2.652151,2.656393,2.471986,1.979627,1.322296,0.450078,-0.460306,-0.960210,-1.105745,...,0.743805,0.496968,0.220211,-0.348993,-0.996680,-1.105747,-1.105752,-0.216993,0.826037,1.606459
5122,2.060756,2.492696,2.652151,2.656393,2.471986,1.979627,1.322296,0.450078,-0.460306,-0.960210,...,1.296787,0.743805,0.496968,0.220211,-0.348993,-0.996680,-1.105747,-1.105752,-0.216993,0.826037
5123,1.288239,2.060756,2.492696,2.652151,2.656393,2.471986,1.979627,1.322296,0.450078,-0.460306,...,1.865043,1.296787,0.743805,0.496968,0.220211,-0.348993,-0.996680,-1.105747,-1.105752,-0.216993


In [43]:
test_x_val = test_x.values

In [44]:
fc_array = []

for x in range(0, len(test_x_val), 14):
    test_nn = test_x_val[x]
#     print(test_nn)
    test_nn = test_nn.reshape(1, 1, test_nn.shape[0])
#     print(test_nn.shape)
    fc = model.predict(test_nn)
    fc = scaler.inverse_transform(fc)
#     print(fc.tolist()[0])
    fc_array.extend(fc.tolist()[0])

In [45]:
len(fc_array)

504

In [50]:
data[-14*36: ]

,power
date_str,
2021-01-09 07:00:00,11.49
2021-01-09 08:00:00,2827.62
2021-01-09 09:00:00,19126.00
2021-01-09 10:00:00,81339.20
2021-01-09 11:00:00,120248.86
...,...
2021-02-13 16:00:00,255891.15
2021-02-13 17:00:00,231299.38
2021-02-13 18:00:00,255489.24


In [54]:
pd.DataFrame(fc_array, index = data[-14*36: ].index, columns = ['fc'])

,fc
date_str,
2021-01-09 07:00:00,-21509.103516
2021-01-09 08:00:00,3088.638428
2021-01-09 09:00:00,53766.593750
2021-01-09 10:00:00,127431.109375
2021-01-09 11:00:00,195122.609375
...,...
2021-02-13 16:00:00,246591.796875
2021-02-13 17:00:00,220467.859375
2021-02-13 18:00:00,156218.750000


In [49]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data[-14*36: ].index, y = data[-14*36: ]['power']))
fig.add_trace(go.Scatter(x=data[-14*36: ].index, y = fc_array))

With weather data

In [97]:
data_pc = pd.read_pickle('../ts_data/6010').iloc[:, 0:8]

In [98]:
data_pc.head()

,power,wind,temperature,dewPoint,cloudCover,humidity,pressure,uvIndex
date_str,,,,,,,,
2020-02-13 07:00:00,7.22,3.20,21.92,19.28,0.21,0.85,1011.2,0
2020-02-13 08:00:00,3.52,3.17,22.55,19.48,0.00,0.83,1011.5,0
2020-02-13 09:00:00,1784.07,3.51,23.09,19.69,0.00,0.81,1011.9,0
2020-02-13 10:00:00,405.33,4.00,23.98,19.83,0.00,0.78,1012.3,0
2020-02-13 11:00:00,16232.33,4.24,25.35,20.25,0.00,0.73,1012.3,2


In [121]:
def get_lags_pc(dataframe):
    dfs = []
    for i in range(1, 14*2 +1):
        df1 = dataframe[['power']].shift(i)
        df1.columns = [col + "_" + str(-i) for col in ['power']]
        dfs.append(df1.iloc[:,0:])
    
    dfs.append(dataframe.iloc[:, 1:])
    for i in range(1, 14):
        df2 = dataframe.iloc[:,1:].shift(-i)
        df2.columns = [col + "_" + str(i) for col in  dataframe.iloc[:,1:].columns]
        dfs.append(df2.iloc[:,0:]) 
    dfs.append(dataframe[['power']])
    new_df = pd.concat(dfs, axis=1).dropna()
    return new_df

In [124]:
nn_pc = get_lags_pc(data_pc)
nn_output = get_output(nn_pc)

In [127]:
train_x , train_y = nn_output.iloc[:, :-14], nn_output.iloc[:, -14:]

In [129]:
train_x

,power_-1,power_-2,power_-3,power_-4,power_-5,power_-6,power_-7,power_-8,power_-9,power_-10,...,humidity_12,pressure_12,uvIndex_12,wind_13,temperature_13,dewPoint_13,cloudCover_13,humidity_13,pressure_13,uvIndex_13
date_str,,,,,,,,,,,,,,,,,,,,,
2020-02-15 07:00:00,8636.24,16497.84,23686.40,28443.07,30289.86,30769.21,30339.29,27991.02,22989.98,15647.94,...,0.93,1009.7,3.0,6.54,23.19,21.82,1.00,0.92,1010.0,2.0
2020-02-15 08:00:00,6.14,8636.24,16497.84,23686.40,28443.07,30289.86,30769.21,30339.29,27991.02,22989.98,...,0.92,1010.0,2.0,5.52,19.71,16.89,0.80,0.84,1012.7,0.0
2020-02-15 09:00:00,3.44,6.14,8636.24,16497.84,23686.40,28443.07,30289.86,30769.21,30339.29,27991.02,...,0.84,1012.7,0.0,5.48,19.42,16.83,0.71,0.85,1013.1,0.0
2020-02-15 10:00:00,521.86,3.44,6.14,8636.24,16497.84,23686.40,28443.07,30289.86,30769.21,30339.29,...,0.85,1013.1,0.0,5.07,19.05,16.76,0.65,0.87,1013.7,0.0
2020-02-15 11:00:00,7987.27,521.86,3.44,6.14,8636.24,16497.84,23686.40,28443.07,30289.86,30769.21,...,0.87,1013.7,0.0,4.55,18.88,16.76,0.59,0.88,1014.7,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-11 18:00:00,30735.79,32494.80,32590.01,31924.83,30190.22,25641.15,18347.17,8990.85,2473.83,1.01,...,0.49,1013.8,12.0,5.28,27.34,15.13,0.00,0.47,1013.0,10.0
2021-02-11 19:00:00,26041.20,30735.79,32494.80,32590.01,31924.83,30190.22,25641.15,18347.17,8990.85,2473.83,...,0.47,1013.0,10.0,6.03,27.18,15.94,0.00,0.50,1012.5,8.0
2021-02-11 20:00:00,18099.72,26041.20,30735.79,32494.80,32590.01,31924.83,30190.22,25641.15,18347.17,8990.85,...,0.50,1012.5,8.0,6.70,26.68,16.64,0.00,0.54,1012.1,5.0


In [130]:
scaler = StandardScaler()
scaler.fit(data_pc.values)

StandardScaler()

In [58]:
def read_loss_curve(model_path, iter_num):
    return pd.read_pickle(f'../{model_path}/training_loss_iteration_{iter_num}')

In [66]:
def plot_loss(loss_array, models):

    fig = go.Figure()
    
    i =0
    for loss in loss_array:
        fig.add_trace(go.Scatter(y= loss['loss'] , name = f'{models[i]}_train-loss' ))
        fig.add_trace(go.Scatter(y= loss['val_loss'], name = f'{models[i]}_val-loss'))
        i +=1
    fig.show()

In [70]:
loss_grid = read_loss_curve('dnn_results/lstm/grid', 1)

In [71]:
plot_loss([loss_grid], ['lstm-grid'])

In [72]:
loss_pc = read_loss_curve('dnn_results/lstm/6014', 1)

In [73]:
plot_loss([loss_pc], ['lstm-pc'])